In [ ]:
import numpy as np
from collections import defaultdict
import random
import copy
from tabulate import tabulate
import traceback

class SubjectTeacherAllocation:
    def __init__(self, teacher_code, num_lectures, assigned_lectures=0):
        self.teacher_code = teacher_code
        self.num_lectures = num_lectures
        self.assigned_lectures = assigned_lectures
        
class BatchSchedule:
    def __init__(self, subject, teacher, batch_num):
        self.subject = subject
        self.teacher = teacher
        self.batch_num = batch_num

class MultiClassTimetableEnvironment:
    def __init__(self, days, periods_per_day, lunch_after_period, classes, 
                 teachers, teacher_names, subjects, class_requirements, num_batches):
        self.days = days
        self.periods_per_day = periods_per_day
        self.lunch_after_period = lunch_after_period
        self.classes = classes
        self.teachers = teachers
        self.teacher_names = teacher_names
        self.subjects = subjects
        self.class_requirements = class_requirements
        self.num_batches = num_batches
        
        # Track teacher allocations and remaining lectures
        self.teacher_allocations = self._initialize_teacher_allocations()
        
        self.max_consecutive_same_subject = 2
        self.preferred_lab_periods = [0, 2]
        
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, classes[0])
        
    def _initialize_teacher_allocations(self):
        allocations = {}
        for class_name, requirements in self.class_requirements.items():
            allocations[class_name] = {}
            for subject, subject_reqs in requirements.items():
                if subject != 'teacher_code':
                    allocations[class_name][subject] = []
                    for teacher_alloc in subject_reqs['teacher_allocations']:
                        teacher_code, num_lectures = teacher_alloc
                        allocations[class_name][subject].append(
                            SubjectTeacherAllocation(teacher_code, num_lectures)
                        )
        return allocations

    def _create_empty_timetable(self):
        """Create an empty timetable with support for multiple subjects per slot"""
        timetables = {}
        for class_name in self.classes:
            # Initialize with empty lists to store multiple subjects/batches
            timetable = np.empty((self.days, self.periods_per_day + 1), dtype=object)
            for day in range(self.days):
                for period in range(self.periods_per_day + 1):
                    timetable[day, period] = []
                timetable[day, self.lunch_after_period] = 'LUNCH'
            timetables[class_name] = timetable
        return timetables

    def _can_schedule_lab(self, day, period, subject, class_name, teacher_code, batch):
        """Check if a lab session can be scheduled"""
        if period >= self.periods_per_day - 1 or period == self.lunch_after_period - 1:
            return False
            
        current_slot = self.timetable[class_name][day, period]
        next_slot = self.timetable[class_name][day, period + 1]
        
        if current_slot == 'LUNCH' or next_slot == 'LUNCH':
            return False
            
        # Check if this batch already has a lab at this time
        for scheduled in current_slot:
            if isinstance(scheduled, BatchSchedule):
                if scheduled.batch_num == batch:
                    return False
                    
        # Check teacher availability for both periods
        if not self._is_teacher_available(day, period, teacher_code) or \
           not self._is_teacher_available(day, period + 1, teacher_code):
            return False
            
        return True

    def _is_teacher_available(self, day, period, teacher_code):
        """Check teacher availability across all classes and batches"""
        for class_name in self.classes:
            slot = self.timetable[class_name][day, period]
            if slot == 'LUNCH':
                continue
                
            for scheduled in slot:
                if isinstance(scheduled, BatchSchedule):
                    if scheduled.teacher == teacher_code:
                        return False
                else:  # Regular subject
                    if scheduled.teacher == teacher_code:
                        return False
        return True

    def _get_available_teacher(self, class_name, subject):
        """Get an available teacher with remaining lectures"""
        allocations = self.teacher_allocations[class_name][subject]
        for alloc in allocations:
            if alloc.assigned_lectures < alloc.num_lectures:
                return alloc.teacher_code
        return None

    def step(self, action):
        day, period, current_class = self.current_position
        subject, is_lab = action
        reward = 0
        done = False
        
        if period == self.lunch_after_period:
            period += 1
            
        if is_lab:
            # Schedule lab sessions for all batches
            success = True
            scheduled_teachers = set()
            
            for batch in range(self.num_batches):
                teacher_code = self._get_available_teacher(current_class, subject)
                
                if teacher_code and teacher_code not in scheduled_teachers:
                    if self._can_schedule_lab(day, period, subject, current_class, teacher_code, batch):
                        # Schedule lab for this batch
                        batch_schedule = BatchSchedule(subject, teacher_code, batch)
                        self.timetable[current_class][day, period].append(batch_schedule)
                        self.timetable[current_class][day, period + 1].append(batch_schedule)
                        
                        # Update teacher allocation
                        self._update_teacher_allocation(current_class, subject, teacher_code)
                        scheduled_teachers.add(teacher_code)
                    else:
                        success = False
                        break
                else:
                    success = False
                    break
                    
            if success:
                reward = 1
                period += 1
            else:
                reward = -1
        else:
            # Schedule regular subject
            teacher_code = self._get_available_teacher(current_class, subject)
            if teacher_code and self._is_teacher_available(day, period, teacher_code):
                self.timetable[current_class][day, period].append(
                    BatchSchedule(subject, teacher_code, None)
                )
                self._update_teacher_allocation(current_class, subject, teacher_code)
                reward = 1
            else:
                reward = -1
                
        # Update position
        class_idx = self.classes.index(current_class)
        class_idx += 1
        if class_idx >= len(self.classes):
            class_idx = 0
            period += 1
            if period >= self.periods_per_day + 1:
                period = 0
                day += 1
        
        if day >= self.days:
            done = True
            if self._check_all_requirements_met():
                reward += 100
                
        self.current_position = (day, period, self.classes[class_idx])
        return self._get_state(), reward, done

    def _update_teacher_allocation(self, class_name, subject, teacher_code):
        """Update the number of lectures assigned to a teacher"""
        for alloc in self.teacher_allocations[class_name][subject]:
            if alloc.teacher_code == teacher_code:
                alloc.assigned_lectures += 1
                break

    def _check_all_requirements_met(self):
        """Check if all teaching requirements have been met"""
        for class_name, subjects in self.teacher_allocations.items():
            for subject, allocations in subjects.items():
                for alloc in allocations:
                    if alloc.assigned_lectures < alloc.num_lectures:
                        return False
        return True

    def display_detailed_timetable(self):
        
        """Display the timetable with detailed information about teachers and batches"""
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        periods = [f'Period {i+1}' for i in range(self.periods_per_day + 1)]
        
        for class_name in self.classes:
            print(f"\n{'='*100}")
            print(f"Timetable for Class {class_name}")
            print(f"{'='*100}")
            
            table_data = []
            for day_idx, day in enumerate(days):
                row = [day]
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day_idx, period]
                    
                    if slot == 'LUNCH':
                        cell = 'LUNCH BREAK'
                    elif slot:
                        cell_parts = []
                        for scheduled in slot:
                            teacher_name = self.teacher_names[scheduled.teacher]
                            if scheduled.batch_num is not None:
                                cell_parts.append(
                                    f"{scheduled.subject} (Batch {scheduled.batch_num + 1})\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                            else:
                                cell_parts.append(
                                    f"{scheduled.subject}\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                        cell = "\n---\n".join(cell_parts)
                    else:
                        cell = '-'
                    row.append(cell)
                table_data.append(row)
            
            print(tabulate(table_data, headers=['Day'] + periods, tablefmt='grid'))
            
            # Display teacher allocation summary
            print(f"\nTeacher Allocation Summary for Class {class_name}:")
            for subject, allocations in self.teacher_allocations[class_name].items():
                print(f"\n{subject}:")
                for alloc in allocations:
                    teacher_name = self.teacher_names[alloc.teacher_code]
                    print(f"- {alloc.teacher_code} ({teacher_name}):")
                    print(f"  Assigned: {alloc.assigned_lectures}/{alloc.num_lectures} lectures")
                    
    def _get_state(self):
    #"""Get current state of the environment"""
        day, period, current_class = self.current_position
        
        # Get remaining lectures for current class
        remaining_lectures = {}
        for subject, allocations in self.teacher_allocations[current_class].items():
            remaining = sum(alloc.num_lectures - alloc.assigned_lectures 
                        for alloc in allocations)
            remaining_lectures[subject] = remaining
        
        return {
            'position': self.current_position,
            'remaining_lectures': remaining_lectures
    }

    def calculate_timetable_score(self):
        """Calculate the score for the current timetable"""
        score = 0
        
        # Award points for each successfully scheduled lecture
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    score += alloc.assigned_lectures
        
        # Penalty for unassigned lectures
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    unassigned = alloc.num_lectures - alloc.assigned_lectures
                    score -= unassigned * 2  # Higher penalty for unassigned lectures
        
        return score

    def reset(self):
        """Reset the environment to initial state"""
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, self.classes[0])
        self.teacher_allocations = self._initialize_teacher_allocations()
        return self._get_state()

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.95, epsilon=0.3):
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.action_space = action_space
    
    def get_state_key(self, state):
        """Convert state to a string key for Q-table"""
        if state and 'position' in state:
            return str(state['position'])
        return "default"
    
    def get_action(self, state, valid_actions):
        """Get action using epsilon-greedy policy"""
        if not valid_actions:
            return None
        
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        state_key = self.get_state_key(state)
        q_values = {action: self.q_table[state_key][action] 
                   for action in valid_actions}
        
        return max(q_values.items(), key=lambda x: x[1])[0]
    
    def update(self, state, action, reward, next_state):
        """Update Q-value for state-action pair"""
        if not state or not next_state or not action:
            return
        
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)
        
        current_q = self.q_table[state_key][action]
        next_max_q = max([self.q_table[next_state_key][a] for a in self.action_space])
        
        new_q = current_q + self.lr * (reward + self.gamma * next_max_q - current_q)
        self.q_table[state_key][action] = new_q

def get_teacher_details():
    """Get teacher details from user input with better subject handling"""
    teachers = {}
    teacher_names = {}
    
    num_teachers = int(input("Number of teachers: "))
    
    for i in range(num_teachers):
        print(f"\nEnter details for teacher {i+1}:")
        code = input("Teacher code (e.g., T001): ")
        name = input("Teacher name: ")
        print("Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): ")
        subjects = [s.strip().upper() for s in input().split(',')]
        
        teachers[code] = subjects
        teacher_names[code] = name
        print(f"Added teacher {code} ({name}) who can teach: {', '.join(subjects)}")
    
    return teachers, teacher_names

def get_class_requirements(classes, subjects, teachers, teacher_names):
    """Get detailed class requirements with fixed teacher assignment validation"""
    class_requirements = {}
    
    for class_name in classes:
        print(f"\nEnter requirements for class {class_name}:")
        class_requirements[class_name] = {}
        
        for subject_name in subjects:
            print(f"\nFor subject {subject_name}:")
            
            # Show available teachers for this subject
            available_teachers = [
                (code, name) for code, name in teacher_names.items()
                if subject_name.upper() in [s.upper() for s in teachers[code]]
            ]
            
            if not available_teachers:
                print(f"Warning: No teachers available for {subject_name}")
                continue
                
            print("\nAvailable teachers for this subject:")
            for code, name in available_teachers:
                print(f"- {code}: {name}")
            
            # Get number of teachers for this subject
            while True:
                try:
                    num_teachers = int(input(f"Number of teachers for {subject_name}: "))
                    if num_teachers <= 0:
                        print("Please enter a positive number")
                        continue
                    if num_teachers > len(available_teachers):
                        print(f"Only {len(available_teachers)} teachers available for this subject")
                        continue
                    break
                except ValueError:
                    print("Please enter a valid number")
            
            teacher_allocations = []
            total_lectures = 0
            
            # Get allocation for each teacher
            assigned_teachers = set()
            for i in range(num_teachers):
                while True:
                    print("\nAvailable teachers:")
                    for code, name in available_teachers:
                        if code not in assigned_teachers:
                            print(f"- {code}: {name}")
                            
                    teacher_code = input(f"Select teacher {i+1} code for {subject_name}: ")
                    
                    # Validate teacher code
                    if teacher_code not in teachers:
                        print("Invalid teacher code")
                        continue
                    if teacher_code in assigned_teachers:
                        print("This teacher is already assigned to this subject")
                        continue
                    if subject_name.upper() not in [s.upper() for s in teachers[teacher_code]]:
                        print(f"Teacher {teacher_code} cannot teach {subject_name}")
                        continue
                    break
                
                while True:
                    try:
                        lectures = int(input(f"Number of lectures for teacher {teacher_code}: "))
                        if lectures <= 0:
                            print("Please enter a positive number")
                            continue
                        if subjects[subject_name]['is_lab'] and lectures % 2 != 0:
                            print("Lab subjects must have even number of lectures")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
                
                total_lectures += lectures
                teacher_allocations.append((teacher_code, lectures))
                assigned_teachers.add(teacher_code)
                print(f"Assigned {lectures} lectures to {teacher_code} ({teacher_names[teacher_code]})")
            
            class_requirements[class_name][subject_name] = {
                'teacher_allocations': teacher_allocations,
                'total_lectures': total_lectures
            }
            
            print(f"\nTotal lectures for {subject_name}: {total_lectures}")
            print("Teacher allocations:")
            for teacher_code, lectures in teacher_allocations:
                print(f"- {teacher_code} ({teacher_names[teacher_code]}): {lectures} lectures")
    
    return class_requirements

def get_input_data():
    """Get all necessary input data from user"""
    print("Enter timetable parameters:")
    days = 5  # Fixed to 5 days
    periods_per_day = int(input("Number of periods per day: "))
    lunch_after_period = int(input("Lunch break after period number: "))
    
    print("\nEnter class details:")
    num_classes = int(input("Number of classes: "))
    classes = []
    for i in range(num_classes):
        class_name = input(f"Name for class {i+1}: ")
        classes.append(class_name)
    
    print("\nEnter subject details:")
    subjects = {}
    num_subjects = int(input("Number of subjects: "))
    for i in range(num_subjects):
        while True:
            subject_name = input(f"Name for subject {i+1}: ").upper()
            if subject_name in subjects:
                print("Subject already exists. Please enter a different name.")
                continue
            break
        is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
        subjects[subject_name] = {'is_lab': is_lab}
    
    num_batches = int(input("\nNumber of lab batches: "))
    
    print("\nEnter teacher details:")
    teachers, teacher_names = get_teacher_details()
    
    class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names)
    
    return (days, periods_per_day, lunch_after_period, classes, teachers, 
            teacher_names, subjects, class_requirements, num_batches)

def train_timetable_generator(env, agent, episodes=1000):
    """Train the timetable generator"""
    best_timetable = None
    best_score = float('-inf')
    
    for episode in range(episodes):
        state = env.reset() if hasattr(env, 'reset') else None
        total_reward = 0
        done = False
        
        while not done:
            if not state:
                break
            
            current_class = state['position'][2]
            valid_actions = []
            
            for action in agent.action_space:
                subject, is_lab = action
                if env._get_available_teacher(current_class, subject):
                    valid_actions.append(action)
            
            if not valid_actions:
                break
            
            action = agent.get_action(state, valid_actions)
            if not action:
                break
            
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            
            state = next_state
            total_reward += reward
        
        score = env.calculate_timetable_score() if hasattr(env, 'calculate_timetable_score') else total_reward
        if score > best_score:
            best_score = score
            best_timetable = copy.deepcopy(env.timetable)
        
        if episode % 100 == 0:
            print(f"Episode {episode}, Best Score: {best_score}")
    
    return best_timetable

def main():
    try:
        # Get all input data
        (days, periods_per_day, lunch_after_period, classes, teachers, 
         teacher_names, subjects, class_requirements, num_batches) = get_input_data()
        
        # Create environment
        env = MultiClassTimetableEnvironment(
            days, periods_per_day, lunch_after_period, classes, 
            teachers, teacher_names, subjects, class_requirements, num_batches
        )
        
        # Create action space
        action_space = []
        for subject, info in subjects.items():
            action_space.append((subject, info['is_lab']))
        
        # Create and train agent
        # In main(), modify the agent creation and training:
        agent = QLearningAgent(action_space, learning_rate=0.1, discount_factor=0.99, epsilon=0.4)
        best_timetable = train_timetable_generator(env, agent, episodes=2000)  # Increase episodes      
        
        if best_timetable is not None:
            env.timetable = best_timetable
            env.display_detailed_timetable()
        else:
            print("Failed to generate a valid timetable. Please try again.")
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()
        
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()

# More changes

In [ ]:
import numpy as np
from collections import defaultdict
import random
import copy
from tabulate import tabulate
import traceback

class SubjectTeacherAllocation:
    def __init__(self, teacher_code, num_lectures, assigned_lectures=0):
        self.teacher_code = teacher_code
        self.num_lectures = num_lectures
        self.assigned_lectures = assigned_lectures

class BatchSchedule:
    def __init__(self, subject, teacher, batch_num):
        self.subject = subject
        self.teacher = teacher
        self.batch_num = batch_num

class MultiClassTimetableEnvironment:
    def __init__(self, days, periods_per_day, lunch_after_period, classes, 
                 teachers, teacher_names, subjects, class_requirements, num_batches):
        self.days = days
        self.periods_per_day = periods_per_day
        self.lunch_after_period = lunch_after_period
        self.classes = classes
        self.teachers = teachers
        self.teacher_names = teacher_names
        self.subjects = subjects
        self.class_requirements = class_requirements
        self.num_batches = num_batches
        
        # Track teacher allocations and remaining lectures
        self.teacher_allocations = self._initialize_teacher_allocations()
        
        self.max_consecutive_same_subject = 2
        self.preferred_lab_periods = [0, 2]
        
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, classes[0])
        
    def _initialize_teacher_allocations(self):
        allocations = {}
        for class_name, requirements in self.class_requirements.items():
            allocations[class_name] = {}
            for subject, subject_reqs in requirements.items():
                if subject != 'teacher_code':
                    allocations[class_name][subject] = []
                    for teacher_alloc in subject_reqs['teacher_allocations']:
                        teacher_code, num_lectures = teacher_alloc
                        allocations[class_name][subject].append(
                            SubjectTeacherAllocation(teacher_code, num_lectures)
                        )
        return allocations

    def _create_empty_timetable(self):
        """Create an empty timetable with support for multiple subjects per slot"""
        timetables = {}
        for class_name in self.classes:
            # Initialize with empty lists to store multiple subjects/batches
            timetable = np.empty((self.days, self.periods_per_day + 1), dtype=object)
            for day in range(self.days):
                for period in range(self.periods_per_day + 1):
                    timetable[day, period] = []
                timetable[day, self.lunch_after_period] = 'LUNCH'
            timetables[class_name] = timetable
        return timetables

    def _can_schedule_lab(self, day, period, subject, class_name):
        """Check if a lab session can be scheduled for all batches"""
        if period >= self.periods_per_day - 1 or period == self.lunch_after_period - 1:
            return False
                
        current_slot = self.timetable[class_name][day, period]
        next_slot = self.timetable[class_name][day, period + 1]
            
        if current_slot == 'LUNCH' or next_slot == 'LUNCH':
            return False
                
        # Check if any batch has a lab at this time
        if current_slot or next_slot:
            return False
                
        # Get available teachers for this lab
        available_teachers = []
        allocations = self.teacher_allocations[class_name][subject]
        
        # For labs, total lectures across all teachers should match number of sessions
        total_assigned = sum(alloc.assigned_lectures for alloc in allocations)
        target_sessions = allocations[0].num_lectures  # All teachers should have same number
        
        if total_assigned >= target_sessions:
            return False
        
        # Check if we have enough available teachers for all batches
        for alloc in allocations:
            if self._is_teacher_available(day, period, alloc.teacher_code) and \
            self._is_teacher_available(day, period + 1, alloc.teacher_code):
                available_teachers.append(alloc.teacher_code)
        
        return len(available_teachers) >= self.num_batches

    def _is_teacher_available(self, day, period, teacher_code):
        """Check teacher availability across all classes and batches"""
        for class_name in self.classes:
            slot = self.timetable[class_name][day, period]
            if slot == 'LUNCH':
                continue
                
            for scheduled in slot:
                if isinstance(scheduled, BatchSchedule):
                    if scheduled.teacher == teacher_code:
                        return False
                else:  # Regular subject
                    if scheduled.teacher == teacher_code:
                        return False
        return True

    def _get_available_teacher(self, class_name, subject):
        """Get an available teacher with remaining lectures"""
        allocations = self.teacher_allocations[class_name][subject]
        for alloc in allocations:
            if alloc.assigned_lectures < alloc.num_lectures:
                return alloc.teacher_code
        return None

    def step(self, action):
        """Take a step in the environment with updated lab/lecture logic"""
        day, period, current_class = self.current_position
        subject, is_lab = action
        reward = 0
        done = False
            
        if period == self.lunch_after_period:
            period += 1
                
        if is_lab:
            # Validate number of teachers equals number of batches
            allocations = self.teacher_allocations[current_class][subject]
            if len(allocations) != self.num_batches:
                reward = -1
            else:
                # Try to schedule lab for all batches
                if self._can_schedule_lab(day, period, subject, current_class):
                    available_teachers = []
                    for alloc in allocations:
                        if self._is_teacher_available(day, period, alloc.teacher_code):
                            available_teachers.append(alloc.teacher_code)
                    
                    if len(available_teachers) >= self.num_batches:
                        # Schedule lab for all batches
                        for batch in range(self.num_batches):
                            teacher_code = available_teachers[batch]
                            batch_schedule = BatchSchedule(subject, teacher_code, batch)
                            self.timetable[current_class][day, period].append(batch_schedule)
                            self.timetable[current_class][day, period + 1].append(batch_schedule)
                        
                        # Update all teachers' allocation counts (shared total)
                        for alloc in allocations:
                            alloc.assigned_lectures += 1
                        
                        reward = 1
                        period += 1
                    else:
                        reward = -1
                else:
                    reward = -1
        else:
            # Regular lecture - whole class together
            teacher_code = self._get_available_teacher(current_class, subject)
            if teacher_code and self._is_teacher_available(day, period, teacher_code):
                # Schedule for whole class (no batch number)
                self.timetable[current_class][day, period].append(
                    BatchSchedule(subject, teacher_code, None)
                )
                self._update_teacher_allocation(current_class, subject, teacher_code)
                reward = 1
            else:
                reward = -1
                    
        # Update position
        class_idx = self.classes.index(current_class)
        class_idx += 1
        if class_idx >= len(self.classes):
            class_idx = 0
            period += 1
            if period >= self.periods_per_day + 1:
                period = 0
                day += 1
            
        if day >= self.days:
            done = True
            if self._check_all_requirements_met():
                reward += 100
                    
        self.current_position = (day, period, self.classes[class_idx])
        return self._get_state(), reward, done

    def _update_teacher_allocation(self, class_name, subject, teacher_code):
        """Update the number of lectures assigned to a teacher"""
        for alloc in self.teacher_allocations[class_name][subject]:
            if alloc.teacher_code == teacher_code:
                alloc.assigned_lectures += 1
                break

    def _check_all_requirements_met(self):
        """Check if all teaching requirements have been met"""
        for class_name, subjects in self.teacher_allocations.items():
            for subject, allocations in subjects.items():
                for alloc in allocations:
                    if alloc.assigned_lectures < alloc.num_lectures:
                        return False
        return True

    def display_detailed_timetable(self):
        
        """Display the timetable with detailed information about teachers and batches"""
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        periods = [f'Period {i+1}' for i in range(self.periods_per_day + 1)]
        
        for class_name in self.classes:
            print(f"\n{'='*100}")
            print(f"Timetable for Class {class_name}")
            print(f"{'='*100}")
            
            table_data = []
            for day_idx, day in enumerate(days):
                row = [day]
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day_idx, period]
                    
                    if slot == 'LUNCH':
                        cell = 'LUNCH BREAK'
                    elif slot:
                        cell_parts = []
                        for scheduled in slot:
                            teacher_name = self.teacher_names[scheduled.teacher]
                            if scheduled.batch_num is not None:
                                cell_parts.append(
                                    f"{scheduled.subject} (Batch {scheduled.batch_num + 1})\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                            else:
                                cell_parts.append(
                                    f"{scheduled.subject}\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                        cell = "\n---\n".join(cell_parts)
                    else:
                        cell = '-'
                    row.append(cell)
                table_data.append(row)
            
            print(tabulate(table_data, headers=['Day'] + periods, tablefmt='grid'))
            
            # Display teacher allocation summary
            print(f"\nTeacher Allocation Summary for Class {class_name}:")
            for subject, allocations in self.teacher_allocations[class_name].items():
                print(f"\n{subject}:")
                for alloc in allocations:
                    teacher_name = self.teacher_names[alloc.teacher_code]
                    print(f"- {alloc.teacher_code} ({teacher_name}):")
                    print(f"  Assigned: {alloc.assigned_lectures}/{alloc.num_lectures} lectures")
                    
    def _get_state(self):
    #"""Get current state of the environment"""
        day, period, current_class = self.current_position
        
        # Get remaining lectures for current class
        remaining_lectures = {}
        for subject, allocations in self.teacher_allocations[current_class].items():
            remaining = sum(alloc.num_lectures - alloc.assigned_lectures 
                        for alloc in allocations)
            remaining_lectures[subject] = remaining
        
        return {
            'position': self.current_position,
            'remaining_lectures': remaining_lectures
    }

    def calculate_timetable_score(self):
        """Calculate the score for the current timetable"""
        score = 0
        
        # Award points for each successfully scheduled lecture
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    score += alloc.assigned_lectures
        
        # Penalty for unassigned lectures
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    unassigned = alloc.num_lectures - alloc.assigned_lectures
                    score -= unassigned * 2  # Higher penalty for unassigned lectures
        
        return score

    def reset(self):
        """Reset the environment to initial state"""
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, self.classes[0])
        self.teacher_allocations = self._initialize_teacher_allocations()
        return self._get_state()

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.95, epsilon=0.3):
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.action_space = action_space
    
    def get_state_key(self, state):
        """Convert state to a string key for Q-table"""
        if state and 'position' in state:
            return str(state['position'])
        return "default"
    
    def get_action(self, state, valid_actions):
        """Get action using epsilon-greedy policy"""
        if not valid_actions:
            return None
        
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        state_key = self.get_state_key(state)
        q_values = {action: self.q_table[state_key][action] 
                   for action in valid_actions}
        
        return max(q_values.items(), key=lambda x: x[1])[0]
    
    def update(self, state, action, reward, next_state):
        """Update Q-value for state-action pair"""
        if not state or not next_state or not action:
            return
        
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)
        
        current_q = self.q_table[state_key][action]
        next_max_q = max([self.q_table[next_state_key][a] for a in self.action_space])
        
        new_q = current_q + self.lr * (reward + self.gamma * next_max_q - current_q)
        self.q_table[state_key][action] = new_q

def get_teacher_details():
    """Get teacher details from user input with better subject handling"""
    teachers = {}
    teacher_names = {}
    
    num_teachers = int(input("Number of teachers: "))
    
    for i in range(num_teachers):
        print(f"\nEnter details for teacher {i+1}:")
        code = input("Teacher code (e.g., T001): ")
        name = input("Teacher name: ")
        print("Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): ")
        subjects = [s.strip().upper() for s in input().split(',')]
        
        teachers[code] = subjects
        teacher_names[code] = name
        print(f"Added teacher {code} ({name}) who can teach: {', '.join(subjects)}")
    
    return teachers, teacher_names

def get_class_requirements(classes, subjects, teachers, teacher_names, num_batches):
    """Modified input validation for lab teacher requirements"""
    class_requirements = {}
    
    for class_name in classes:
        print(f"\nEnter requirements for class {class_name}:")
        class_requirements[class_name] = {}
        
        for subject_name, subject_info in subjects.items():
            print(f"\nFor subject {subject_name}:")
            
            # Show available teachers for this subject
            available_teachers = [
                (code, name) for code, name in teacher_names.items()
                if subject_name.upper() in [s.upper() for s in teachers[code]]
            ]
            
            if not available_teachers:
                print(f"Warning: No teachers available for {subject_name}")
                continue
                
            print("\nAvailable teachers for this subject:")
            for code, name in available_teachers:
                print(f"- {code}: {name}")
            
            # For labs, enforce number of teachers equals number of batches
            if subject_info['is_lab']:
                num_teachers = num_batches
                print(f"Lab subject - Number of teachers set to number of batches ({num_batches})")
            else:
                while True:
                    try:
                        num_teachers = int(input(f"Number of teachers for {subject_name}: "))
                        if num_teachers <= 0:
                            print("Please enter a positive number")
                            continue
                        if num_teachers > len(available_teachers):
                            print(f"Only {len(available_teachers)} teachers available")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
            
            teacher_allocations = []
            total_lectures = 0
            
            # Get allocation for each teacher
            assigned_teachers = set()
            for i in range(num_teachers):
                while True:
                    print("\nAvailable teachers:")
                    for code, name in available_teachers:
                        if code not in assigned_teachers:
                            print(f"- {code}: {name}")
                            
                    teacher_code = input(f"Select teacher {i+1} code for {subject_name}: ")
                    
                    if teacher_code not in teachers:
                        print("Invalid teacher code")
                        continue
                    if teacher_code in assigned_teachers:
                        print("This teacher is already assigned")
                        continue
                    if subject_name.upper() not in [s.upper() for s in teachers[teacher_code]]:
                        print(f"Teacher {teacher_code} cannot teach {subject_name}")
                        continue
                    break
                
                while True:
                    try:
                        if subject_info['is_lab']:
                            if i == 0:  # Only ask once for labs
                                print("For labs, enter total number of lab sessions")
                                print("(This will be shared among all teachers)")
                                lectures = int(input(f"Number of lab sessions: "))
                            # All teachers get the same number for labs
                        else:
                            lectures = int(input(f"Number of lectures for teacher {teacher_code}: "))
                        
                        if lectures <= 0:
                            print("Please enter a positive number")
                            continue
                        if subject_info['is_lab'] and lectures % 2 != 0:
                            print("Lab sessions must be even number")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
                
                teacher_allocations.append((teacher_code, lectures))
                assigned_teachers.add(teacher_code)
                
                if subject_info['is_lab']:
                    # For labs, all teachers share the same total
                    total_lectures = lectures
                else:
                    total_lectures += lectures
                
            class_requirements[class_name][subject_name] = {
                'teacher_allocations': teacher_allocations,
                'total_lectures': total_lectures
            }
            
            print(f"\nTotal lectures for {subject_name}: {total_lectures}")
            print("Teacher allocations:")
            for teacher_code, lectures in teacher_allocations:
                print(f"- {teacher_code} ({teacher_names[teacher_code]}): "
                      f"{lectures} {'sessions' if subject_info['is_lab'] else 'lectures'}")
    
    return class_requirements

def get_input_data():
    """Get all necessary input data from user"""
    print("Enter timetable parameters:")
    days = 5  # Fixed to 5 days
    periods_per_day = int(input("Number of periods per day: "))
    lunch_after_period = int(input("Lunch break after period number: "))
    
    print("\nEnter class details:")
    num_classes = int(input("Number of classes: "))
    classes = []
    for i in range(num_classes):
        class_name = input(f"Name for class {i+1}: ")
        classes.append(class_name)
    
    print("\nEnter subject details:")
    subjects = {}
    num_subjects = int(input("Number of subjects: "))
    for i in range(num_subjects):
        while True:
            subject_name = input(f"Name for subject {i+1}: ").upper()
            if subject_name in subjects:
                print("Subject already exists. Please enter a different name.")
                continue
            break
        is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
        subjects[subject_name] = {'is_lab': is_lab}
    
    num_batches = int(input("\nNumber of lab batches: "))
    
    print("\nEnter teacher details:")
    teachers, teacher_names = get_teacher_details()
    
    class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names, num_batches)
    
    return (days, periods_per_day, lunch_after_period, classes, teachers, 
            teacher_names, subjects, class_requirements, num_batches)

def train_timetable_generator(env, agent, episodes=1000):
    """Train the timetable generator"""
    best_timetable = None
    best_score = float('-inf')
    
    for episode in range(episodes):
        state = env.reset() if hasattr(env, 'reset') else None
        total_reward = 0
        done = False
        
        while not done:
            if not state:
                break
            
            current_class = state['position'][2]
            valid_actions = []
            
            for action in agent.action_space:
                subject, is_lab = action
                if env._get_available_teacher(current_class, subject):
                    valid_actions.append(action)
            
            if not valid_actions:
                break
            
            action = agent.get_action(state, valid_actions)
            if not action:
                break
            
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            
            state = next_state
            total_reward += reward
        
        score = env.calculate_timetable_score() if hasattr(env, 'calculate_timetable_score') else total_reward
        if score > best_score:
            best_score = score
            best_timetable = copy.deepcopy(env.timetable)
        
        if episode % 100 == 0:
            print(f"Episode {episode}, Best Score: {best_score}")
    
    return best_timetable

def main():
    try:
        # Get all input data
        (days, periods_per_day, lunch_after_period, classes, teachers, 
         teacher_names, subjects, class_requirements, num_batches) = get_input_data()
        
        # Create environment
        env = MultiClassTimetableEnvironment(
            days, periods_per_day, lunch_after_period, classes, 
            teachers, teacher_names, subjects, class_requirements, num_batches
        )
        
        # Create action space
        action_space = []
        for subject, info in subjects.items():
            action_space.append((subject, info['is_lab']))
        
        # Create and train agent
        # In main(), modify the agent creation and training:
        agent = QLearningAgent(action_space, learning_rate=0.1, discount_factor=0.99, epsilon=0.4)
        best_timetable = train_timetable_generator(env, agent, episodes=2000)  # Increase episodes      
        
        if best_timetable is not None:
            env.timetable = best_timetable
            env.display_detailed_timetable()
        else:
            print("Failed to generate a valid timetable. Please try again.")
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()
        
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        traceback.print_exc()

# Enhanced code

In [10]:
import numpy as np
from collections import defaultdict
import random
import copy
from tabulate import tabulate
import traceback
import json
from datetime import datetime
import logging
from typing import Dict, List, Tuple, Optional, Any
import concurrent.futures
from dataclasses import dataclass
import pickle

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

@dataclass
class SubjectTeacherAllocation:
    teacher_code: str
    num_lectures: int
    assigned_lectures: int = 0
    
    def is_fully_allocated(self) -> bool:
        return self.assigned_lectures >= self.num_lectures

@dataclass
class BatchSchedule:
    subject: str
    teacher: str
    batch_num: Optional[int]
    
    def to_dict(self) -> Dict:
        return {
            'subject': self.subject,
            'teacher': self.teacher,
            'batch_num': self.batch_num
        }

class TimetableConstraints:
    def __init__(self):
        self.max_consecutive_same_subject = 2
        self.max_daily_lectures_per_teacher = 6
        self.preferred_lab_periods = [0, 2]
        self.min_gap_between_same_subject = 1

class MultiClassTimetableEnvironment:
    def __init__(self, days: int, periods_per_day: int, lunch_after_period: int,
                 classes: List[str], teachers: Dict[str, List[str]], 
                 teacher_names: Dict[str, str], subjects: Dict[str, Dict],
                 class_requirements: Dict[str, Dict], num_batches: int):
        self.days = days
        self.periods_per_day = periods_per_day
        self.lunch_after_period = lunch_after_period
        self.classes = classes
        self.teachers = teachers
        self.teacher_names = teacher_names
        self.subjects = subjects
        self.class_requirements = class_requirements
        self.num_batches = num_batches
        self.constraints = TimetableConstraints()
        
        # Initialize metrics tracking
        self.metrics = {
            'teacher_utilization': defaultdict(int),
            'subject_distribution': defaultdict(lambda: defaultdict(int)),
            'conflicts_avoided': 0
        }
        
        self.teacher_allocations = self._initialize_teacher_allocations()
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, classes[0])

    def load_timetable(self, filename: str):
        """Load a timetable state from a file"""
        with open(filename, 'r') as f:
            data = json.load(f)
            
        # Reconstruct timetable with proper objects
        for class_name, schedule in data['timetable'].items():
            for pos, slot in schedule.items():
                day, period = eval(pos)
                if isinstance(slot, list):
                    self.timetable[class_name][day, period] = [
                        BatchSchedule(**s) if isinstance(s, dict) else s
                        for s in slot
                    ]
                else:
                    self.timetable[class_name][day, period] = slot
                    
        self.metrics.update(data['metrics'])
        logging.info(f"Timetable loaded from {filename}")

    def _check_teacher_workload(self, day: int, teacher_code: str) -> bool:
        """Check if teacher has exceeded daily workload limit"""
        daily_lectures = 0
        for class_name in self.classes:
            for period in range(self.periods_per_day + 1):
                slot = self.timetable[class_name][day, period]
                if isinstance(slot, list):
                    for scheduled in slot:
                        if isinstance(scheduled, BatchSchedule) and scheduled.teacher == teacher_code:
                            daily_lectures += 1
        return daily_lectures < self.constraints.max_daily_lectures_per_teacher

    def _check_subject_spacing(self, day: int, period: int, subject: str, class_name: str) -> bool:
        """Check if subject spacing constraints are met"""
        # Check previous periods
        for p in range(max(0, period - self.constraints.min_gap_between_same_subject), period):
            slot = self.timetable[class_name][day, p]
            if isinstance(slot, list) and any(s.subject == subject for s in slot):
                return False
                
        # Check following periods
        for p in range(period + 1, 
                      min(self.periods_per_day + 1, 
                          period + self.constraints.min_gap_between_same_subject + 1)):
            slot = self.timetable[class_name][day, p]
            if isinstance(slot, list) and any(s.subject == subject for s in slot):
                return False
        return True

    def calculate_metrics(self) -> Dict[str, Any]:
        """Calculate comprehensive metrics for the timetable"""
        metrics = {
            'teacher_utilization': defaultdict(float),
            'subject_distribution': defaultdict(lambda: defaultdict(int)),
            'gaps_analysis': defaultdict(int),
            'constraint_violations': 0
        }
        
        for class_name in self.classes:
            for day in range(self.days):
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day, period]
                    if isinstance(slot, list) and slot:
                        for scheduled in slot:
                            if isinstance(scheduled, BatchSchedule):
                                metrics['teacher_utilization'][scheduled.teacher] += 1
                                metrics['subject_distribution'][class_name][scheduled.subject] += 1
                                
        # Calculate percentage utilization
        total_periods = self.days * self.periods_per_day
        for teacher in metrics['teacher_utilization']:
            metrics['teacher_utilization'][teacher] = (
                metrics['teacher_utilization'][teacher] / total_periods * 100
            )
            
        return metrics

    def _initialize_teacher_allocations(self) -> Dict[str, Dict[str, List[SubjectTeacherAllocation]]]:
        """Initialize teacher allocations for each class and subject"""
        allocations = {}
        
        for class_name, requirements in self.class_requirements.items():
            allocations[class_name] = {}
            
            for subject, subject_reqs in requirements.items():
                if subject == 'teacher_code':  # Skip non-subject keys
                    continue
                    
                allocations[class_name][subject] = []
                teacher_allocs = subject_reqs.get('teacher_allocations', [])
                
                for teacher_code, num_lectures in teacher_allocs:
                    allocation = SubjectTeacherAllocation(
                        teacher_code=teacher_code,
                        num_lectures=num_lectures,
                        assigned_lectures=0
                    )
                    allocations[class_name][subject].append(allocation)
                    
                # Log initialization for debugging
                logging.debug(f"Initialized allocations for {class_name} - {subject}: "
                            f"{len(allocations[class_name][subject])} teachers")
        
        return allocations

    def _create_empty_timetable(self):
        """Create an empty timetable with support for multiple subjects per slot"""
        timetables = {}
        for class_name in self.classes:
            # Initialize with empty lists to store multiple subjects/batches
            timetable = np.empty((self.days, self.periods_per_day + 1), dtype=object)
            for day in range(self.days):
                for period in range(self.periods_per_day + 1):
                    timetable[day, period] = []
                    if period == self.lunch_after_period:
                        timetable[day, period] = 'LUNCH'
            timetables[class_name] = timetable
        return timetables

    def reset(self):
        """Reset the environment to initial state"""
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, self.classes[0])
        self.teacher_allocations = self._initialize_teacher_allocations()
        return self._get_state()
    
    def _can_schedule_lab(self, day, period, subject, class_name):
        """Check if a lab session can be scheduled for all batches"""
        if period >= self.periods_per_day - 1 or period == self.lunch_after_period - 1:
            return False
                
        current_slot = self.timetable[class_name][day, period]
        next_slot = self.timetable[class_name][day, period + 1]
            
        if current_slot == 'LUNCH' or next_slot == 'LUNCH':
            return False
                
        # Check if any batch has a lab at this time
        if current_slot or next_slot:
            return False
                
        # Get available teachers for this lab
        available_teachers = []
        allocations = self.teacher_allocations[class_name][subject]
        
        # For labs, total lectures across all teachers should match number of sessions
        total_assigned = sum(alloc.assigned_lectures for alloc in allocations)
        target_sessions = allocations[0].num_lectures  # All teachers should have same number
        
        if total_assigned >= target_sessions:
            return False
        
        # Check if we have enough available teachers for all batches
        for alloc in allocations:
            if self._is_teacher_available(day, period, alloc.teacher_code) and \
            self._is_teacher_available(day, period + 1, alloc.teacher_code):
                available_teachers.append(alloc.teacher_code)
        
        return len(available_teachers) >= self.num_batches

    def _is_teacher_available(self, day, period, teacher_code):
        """Check teacher availability across all classes and batches"""
        for class_name in self.classes:
            slot = self.timetable[class_name][day, period]
            if slot == 'LUNCH':
                continue
                
            for scheduled in slot:
                if isinstance(scheduled, BatchSchedule):
                    if scheduled.teacher == teacher_code:
                        return False
                else:  # Regular subject
                    if scheduled.teacher == teacher_code:
                        return False
        return True

    def _get_available_teacher(self, class_name, subject):
        """Get an available teacher with remaining lectures"""
        allocations = self.teacher_allocations[class_name][subject]
        for alloc in allocations:
            if alloc.assigned_lectures < alloc.num_lectures:
                return alloc.teacher_code
        return None

    def step(self, action):
        """Take a step in the environment with updated lab/lecture logic"""
        day, period, current_class = self.current_position
        subject, is_lab = action
        reward = 0
        done = False
            
        if period == self.lunch_after_period:
            period += 1
                
        if is_lab:
            # Validate number of teachers equals number of batches
            allocations = self.teacher_allocations[current_class][subject]
            if len(allocations) != self.num_batches:
                reward = -1
            else:
                # Try to schedule lab for all batches
                if self._can_schedule_lab(day, period, subject, current_class):
                    available_teachers = []
                    for alloc in allocations:
                        if self._is_teacher_available(day, period, alloc.teacher_code):
                            available_teachers.append(alloc.teacher_code)
                    
                    if len(available_teachers) >= self.num_batches:
                        # Schedule lab for all batches
                        for batch in range(self.num_batches):
                            teacher_code = available_teachers[batch]
                            batch_schedule = BatchSchedule(subject, teacher_code, batch)
                            self.timetable[current_class][day, period].append(batch_schedule)
                            self.timetable[current_class][day, period + 1].append(batch_schedule)
                        
                        # Update all teachers' allocation counts (shared total)
                        for alloc in allocations:
                            alloc.assigned_lectures += 1
                        
                        reward = 1
                        period += 1
                    else:
                        reward = -1
                else:
                    reward = -1
        else:
            # Regular lecture - whole class together
            teacher_code = self._get_available_teacher(current_class, subject)
            if teacher_code and self._is_teacher_available(day, period, teacher_code):
                # Schedule for whole class (no batch number)
                self.timetable[current_class][day, period].append(
                    BatchSchedule(subject, teacher_code, None)
                )
                self._update_teacher_allocation(current_class, subject, teacher_code)
                reward = 1
            else:
                reward = -1
                    
        # Update position
        class_idx = self.classes.index(current_class)
        class_idx += 1
        if class_idx >= len(self.classes):
            class_idx = 0
            period += 1
            if period >= self.periods_per_day + 1:
                period = 0
                day += 1
            
        if day >= self.days:
            done = True
            if self._check_all_requirements_met():
                reward += 100
                    
        self.current_position = (day, period, self.classes[class_idx])
        return self._get_state(), reward, done

    def _update_teacher_allocation(self, class_name, subject, teacher_code):
        """Update the number of lectures assigned to a teacher"""
        for alloc in self.teacher_allocations[class_name][subject]:
            if alloc.teacher_code == teacher_code:
                alloc.assigned_lectures += 1
                break

    def _check_all_requirements_met(self):
        """Check if all teaching requirements have been met"""
        for class_name, subjects in self.teacher_allocations.items():
            for subject, allocations in subjects.items():
                for alloc in allocations:
                    if alloc.assigned_lectures < alloc.num_lectures:
                        return False
        return True

    def display_detailed_timetable(self):
        
        """Display the timetable with detailed information about teachers and batches"""
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        periods = [f'Period {i+1}' for i in range(self.periods_per_day + 1)]
        
        for class_name in self.classes:
            print(f"\n{'='*100}")
            print(f"Timetable for Class {class_name}")
            print(f"{'='*100}")
            
            table_data = []
            for day_idx, day in enumerate(days):
                row = [day]
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day_idx, period]
                    
                    if slot == 'LUNCH':
                        cell = 'LUNCH BREAK'
                    elif slot:
                        cell_parts = []
                        for scheduled in slot:
                            teacher_name = self.teacher_names[scheduled.teacher]
                            if scheduled.batch_num is not None:
                                cell_parts.append(
                                    f"{scheduled.subject} (Batch {scheduled.batch_num + 1})\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                            else:
                                cell_parts.append(
                                    f"{scheduled.subject}\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                        cell = "\n---\n".join(cell_parts)
                    else:
                        cell = '-'
                    row.append(cell)
                table_data.append(row)
            
            print(tabulate(table_data, headers=['Day'] + periods, tablefmt='grid'))
            
            # Display teacher allocation summary
            print(f"\nTeacher Allocation Summary for Class {class_name}:")
            for subject, allocations in self.teacher_allocations[class_name].items():
                print(f"\n{subject}:")
                for alloc in allocations:
                    teacher_name = self.teacher_names[alloc.teacher_code]
                    print(f"- {alloc.teacher_code} ({teacher_name}):")
                    print(f"  Assigned: {alloc.assigned_lectures}/{alloc.num_lectures} lectures")
                    
    def _get_state(self):
    #"""Get current state of the environment"""
        day, period, current_class = self.current_position
        
        # Get remaining lectures for current class
        remaining_lectures = {}
        for subject, allocations in self.teacher_allocations[current_class].items():
            remaining = sum(alloc.num_lectures - alloc.assigned_lectures 
                        for alloc in allocations)
            remaining_lectures[subject] = remaining
        
        return {
            'position': self.current_position,
            'remaining_lectures': remaining_lectures
    }

    def calculate_timetable_score(self):
        """Calculate the score for the current timetable"""
        score = 0
        
        # Award points for each successfully scheduled lecture
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    score += alloc.assigned_lectures
        
        # Penalty for unassigned lectures
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    unassigned = alloc.num_lectures - alloc.assigned_lectures
                    score -= unassigned * 2  # Higher penalty for unassigned lectures
        
        return score
        
        # Fix for the JSON saving method
    def save_timetable(self, filename: str):
        """Save the current timetable state to a file"""
        try:
            data = {
                'timetable': {},
                'metrics': dict(self.metrics),
                'timestamp': datetime.now().isoformat()
            }
            
            # Convert timetable data to serializable format
            for class_name in self.classes:
                data['timetable'][class_name] = {}
                for day in range(self.days):
                    for period in range(self.periods_per_day + 1):
                        slot = self.timetable[class_name][day, period]
                        key = f"{day},{period}"
                        
                        if slot == 'LUNCH':
                            data['timetable'][class_name][key] = 'LUNCH'
                        elif isinstance(slot, list):
                            data['timetable'][class_name][key] = [
                                {
                                    'subject': s.subject,
                                    'teacher': s.teacher,
                                    'batch_num': s.batch_num
                                } for s in slot if isinstance(s, BatchSchedule)
                            ]
                        else:
                            data['timetable'][class_name][key] = []
            
            with open(filename, 'w') as f:
                json.dump(data, f, indent=4)
            logging.info(f"Timetable saved to {filename}")
            
        except Exception as e:
            logging.error(f"Error saving timetable: {str(e)}")
            raise

    # Fix for the Excel export method
    def export_to_excel(self, filename: str):
        """Export timetable to Excel format"""
        try:
            import pandas as pd
            
            days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
            periods = [f'Period {p+1}' for p in range(self.periods_per_day + 1)]
            
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Export timetable for each class
                for class_name in self.classes:
                    data = []
                    for day_idx, day in enumerate(days):
                        row = []
                        for period in range(self.periods_per_day + 1):
                            slot = self.timetable[class_name][day_idx, period]
                            if slot == 'LUNCH':
                                cell = 'LUNCH BREAK'
                            elif isinstance(slot, list) and slot:
                                cell_parts = []
                                for scheduled in slot:
                                    if isinstance(scheduled, BatchSchedule):
                                        teacher_name = self.teacher_names.get(scheduled.teacher, scheduled.teacher)
                                        if scheduled.batch_num is not None:
                                            cell_parts.append(
                                                f"{scheduled.subject} (B{scheduled.batch_num + 1})\n"
                                                f"{teacher_name}"
                                            )
                                        else:
                                            cell_parts.append(
                                                f"{scheduled.subject}\n{teacher_name}"
                                            )
                                cell = '\n'.join(cell_parts)
                            else:
                                cell = '-'
                            row.append(cell)
                        data.append(row)
                    
                    # Create DataFrame and write to Excel
                    df = pd.DataFrame(data, index=days, columns=periods)
                    df.to_excel(writer, sheet_name=f'Class_{class_name}')
                    
                    # Auto-adjust column widths
                    worksheet = writer.sheets[f'Class_{class_name}']
                    for idx, col in enumerate(df.columns):
                        max_length = max(
                            df[col].astype(str).apply(len).max(),
                            len(str(col))
                        )
                        worksheet.column_dimensions[chr(65 + idx + 1)].width = max_length + 2
                
                # Export metrics on a separate sheet
                metrics = self.calculate_metrics()
                metrics_df = pd.DataFrame([metrics])
                metrics_df.to_excel(writer, sheet_name='Metrics')
            
            logging.info(f"Timetable exported to {filename}")
            
        except ImportError:
            logging.error("Pandas is required for Excel export. Please install it first.")
            raise
        except Exception as e:
            logging.error(f"Error exporting to Excel: {str(e)}")
            raise

    # Also add these imports at the top of the file if not already present
    def get_input_data():
        """Get all necessary input data from user"""
        print("Enter timetable parameters:")
        days = 5  # Fixed to 5 days
        periods_per_day = int(input("Number of periods per day: "))
        lunch_after_period = int(input("Lunch break after period number: "))
        
        print("\nEnter class details:")
        num_classes = int(input("Number of classes: "))
        classes = []
        for i in range(num_classes):
            class_name = input(f"Name for class {i+1}: ")
            classes.append(class_name)
        
        print("\nEnter subject details:")
        subjects = {}
        num_subjects = int(input("Number of subjects: "))
        for i in range(num_subjects):
            while True:
                subject_name = input(f"Name for subject {i+1}: ").upper()
                if subject_name in subjects:
                    print("Subject already exists. Please enter a different name.")
                    continue
                break
            is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
            subjects[subject_name] = {'is_lab': is_lab}
        
        num_batches = int(input("\nNumber of lab batches: "))
        
        print("\nEnter teacher details:")
        teachers, teacher_names = get_teacher_details()
        
        class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names, num_batches)
        
        return (days, periods_per_day, lunch_after_period, classes, teachers, 
                teacher_names, subjects, class_requirements, num_batches)
    
class ParallelQLearningAgent(QLearningAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_workers = 4
        
    def parallel_update(self, experiences):
        """Update Q-values in parallel"""
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            futures = []
            for state, action, reward, next_state in experiences:
                futures.append(
                    executor.submit(self.update, state, action, reward, next_state)
                )
            concurrent.futures.wait(futures)

def get_input_data_from_file(filename: str) -> Tuple:
    """Load input data from JSON file"""
    with open(filename, 'r') as f:
        data = json.load(f)
    
    required_fields = ['days', 'periods_per_day', 'lunch_after_period', 'classes',
                      'teachers', 'teacher_names', 'subjects', 'class_requirements',
                      'num_batches']
                      
    for field in required_fields:
        if field not in data:
            raise ValueError(f"Missing required field: {field}")
            
    return (data['days'], data['periods_per_day'], data['lunch_after_period'],
            data['classes'], data['teachers'], data['teacher_names'],
            data['subjects'], data['class_requirements'], data['num_batches'])

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.95, epsilon=0.3):
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.action_space = action_space
    
    def get_state_key(self, state):
        """Convert state to a string key for Q-table"""
        if state and 'position' in state:
            return str(state['position'])
        return "default"
    
    def get_action(self, state, valid_actions):
        """Get action using epsilon-greedy policy"""
        if not valid_actions:
            return None
        
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        state_key = self.get_state_key(state)
        q_values = {action: self.q_table[state_key][action] 
                   for action in valid_actions}
        
        return max(q_values.items(), key=lambda x: x[1])[0]
    
    def update(self, state, action, reward, next_state):
        """Update Q-value for state-action pair"""
        if not state or not next_state or not action:
            return
        
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)
        
        current_q = self.q_table[state_key][action]
        next_max_q = max([self.q_table[next_state_key][a] for a in self.action_space])
        
        new_q = current_q + self.lr * (reward + self.gamma * next_max_q - current_q)
        self.q_table[state_key][action] = new_q

def get_teacher_details():
    """Get teacher details from user input with better subject handling"""
    teachers = {}
    teacher_names = {}
    
    num_teachers = int(input("Number of teachers: "))
    
    for i in range(num_teachers):
        print(f"\nEnter details for teacher {i+1}:")
        code = input("Teacher code (e.g., T001): ")
        name = input("Teacher name: ")
        print("Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): ")
        subjects = [s.strip().upper() for s in input().split(',')]
        
        teachers[code] = subjects
        teacher_names[code] = name
        print(f"Added teacher {code} ({name}) who can teach: {', '.join(subjects)}")
    
    return teachers, teacher_names

def get_class_requirements(classes, subjects, teachers, teacher_names, num_batches):
    """Modified input validation for lab teacher requirements"""
    class_requirements = {}
    
    for class_name in classes:
        print(f"\nEnter requirements for class {class_name}:")
        class_requirements[class_name] = {}
        
        for subject_name, subject_info in subjects.items():
            print(f"\nFor subject {subject_name}:")
            
            # Show available teachers for this subject
            available_teachers = [
                (code, name) for code, name in teacher_names.items()
                if subject_name.upper() in [s.upper() for s in teachers[code]]
            ]
            
            if not available_teachers:
                print(f"Warning: No teachers available for {subject_name}")
                continue
                
            print("\nAvailable teachers for this subject:")
            for code, name in available_teachers:
                print(f"- {code}: {name}")
            
            # For labs, enforce number of teachers equals number of batches
            if subject_info['is_lab']:
                num_teachers = num_batches
                print(f"Lab subject - Number of teachers set to number of batches ({num_batches})")
            else:
                while True:
                    try:
                        num_teachers = int(input(f"Number of teachers for {subject_name}: "))
                        if num_teachers <= 0:
                            print("Please enter a positive number")
                            continue
                        if num_teachers > len(available_teachers):
                            print(f"Only {len(available_teachers)} teachers available")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
            
            teacher_allocations = []
            total_lectures = 0
            
            # Get allocation for each teacher
            assigned_teachers = set()
            for i in range(num_teachers):
                while True:
                    print("\nAvailable teachers:")
                    for code, name in available_teachers:
                        if code not in assigned_teachers:
                            print(f"- {code}: {name}")
                            
                    teacher_code = input(f"Select teacher {i+1} code for {subject_name}: ")
                    
                    if teacher_code not in teachers:
                        print("Invalid teacher code")
                        continue
                    if teacher_code in assigned_teachers:
                        print("This teacher is already assigned")
                        continue
                    if subject_name.upper() not in [s.upper() for s in teachers[teacher_code]]:
                        print(f"Teacher {teacher_code} cannot teach {subject_name}")
                        continue
                    break
                
                while True:
                    try:
                        if subject_info['is_lab']:
                            if i == 0:  # Only ask once for labs
                                print("For labs, enter total number of lab sessions")
                                print("(This will be shared among all teachers)")
                                lectures = int(input(f"Number of lab sessions: "))
                            # All teachers get the same number for labs
                        else:
                            lectures = int(input(f"Number of lectures for teacher {teacher_code}: "))
                        
                        if lectures <= 0:
                            print("Please enter a positive number")
                            continue
                        if subject_info['is_lab'] and lectures % 2 != 0:
                            print("Lab sessions must be even number")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
                
                teacher_allocations.append((teacher_code, lectures))
                assigned_teachers.add(teacher_code)
                
                if subject_info['is_lab']:
                    # For labs, all teachers share the same total
                    total_lectures = lectures
                else:
                    total_lectures += lectures
                
            class_requirements[class_name][subject_name] = {
                'teacher_allocations': teacher_allocations,
                'total_lectures': total_lectures
            }
            
            print(f"\nTotal lectures for {subject_name}: {total_lectures}")
            print("Teacher allocations:")
            for teacher_code, lectures in teacher_allocations:
                print(f"- {teacher_code} ({teacher_names[teacher_code]}): "
                      f"{lectures} {'sessions' if subject_info['is_lab'] else 'lectures'}")
    
    return class_requirements

def get_input_data():
    """Get all necessary input data from user"""
    print("Enter timetable parameters:")
    days = 5  # Fixed to 5 days
    periods_per_day = int(input("Number of periods per day: "))
    lunch_after_period = int(input("Lunch break after period number: "))
    
    print("\nEnter class details:")
    num_classes = int(input("Number of classes: "))
    classes = []
    for i in range(num_classes):
        class_name = input(f"Name for class {i+1}: ")
        classes.append(class_name)
    
    print("\nEnter subject details:")
    subjects = {}
    num_subjects = int(input("Number of subjects: "))
    for i in range(num_subjects):
        while True:
            subject_name = input(f"Name for subject {i+1}: ").upper()
            if subject_name in subjects:
                print("Subject already exists. Please enter a different name.")
                continue
            break
        is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
        subjects[subject_name] = {'is_lab': is_lab}
    
    num_batches = int(input("\nNumber of lab batches: "))
    
    print("\nEnter teacher details:")
    teachers, teacher_names = get_teacher_details()
    
    class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names, num_batches)
    
    return (days, periods_per_day, lunch_after_period, classes, teachers, 
            teacher_names, subjects, class_requirements, num_batches)

def train_timetable_generator(env, agent, episodes=1000):
    """Train the timetable generator"""
    best_timetable = None
    best_score = float('-inf')
    
    for episode in range(episodes):
        state = env.reset() if hasattr(env, 'reset') else None
        total_reward = 0
        done = False
        
        while not done:
            if not state:
                break
            
            current_class = state['position'][2]
            valid_actions = []
            
            for action in agent.action_space:
                subject, is_lab = action
                if env._get_available_teacher(current_class, subject):
                    valid_actions.append(action)
            
            if not valid_actions:
                break
            
            action = agent.get_action(state, valid_actions)
            if not action:
                break
            
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            
            state = next_state
            total_reward += reward
        
        score = env.calculate_timetable_score() if hasattr(env, 'calculate_timetable_score') else total_reward
        if score > best_score:
            best_score = score
            best_timetable = copy.deepcopy(env.timetable)
        
        if episode % 100 == 0:
            print(f"Episode {episode}, Best Score: {best_score}")
    
    return best_timetable

def main():
    try:
        # Set up logging to both file and console
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('timetable_generator.log'),
                logging.StreamHandler()  # This will show logs in console
            ]
        )
        
        # Get input data
        try:
            input_data = get_input_data_from_file('input_data.json')
            logging.info("Input data loaded from file")
        except FileNotFoundError:
            logging.info("No input file found, getting data from user input")
            input_data = get_input_data()
            
        # Create environment
        env = MultiClassTimetableEnvironment(*input_data)
        
        # Create action space
        action_space = [
            (subject, info['is_lab'])
            for subject, info in input_data[6].items()  # subjects dict
        ]
        
        # Create and train agent with improved parameters
        agent = ParallelQLearningAgent(
            action_space,
            learning_rate=0.15,
            discount_factor=0.99,
            epsilon=0.4
        )
        
        # Train with periodic saving
        best_timetable = None
        best_score = float('-inf')
        
        num_episodes = 3000
        save_interval = 500
        
        for episode in range(num_episodes):
            if episode % save_interval == 0:
                print(f"Training Episode {episode}/{num_episodes}...")
            
            state = env.reset()
            experiences = []
            total_reward = 0
            done = False
            
            while not done:
                if not state:
                    break
                    
                current_class = state['position'][2]
                valid_actions = [
                    action for action in agent.action_space
                    if env._get_available_teacher(current_class, action[0])
                ]
                
                if not valid_actions:
                    break
                    
                action = agent.get_action(state, valid_actions)
                if not action:
                    break
                    
                next_state, reward, done = env.step(action)
                experiences.append((state, action, reward, next_state))
                
                state = next_state
                total_reward += reward
            
            # Update Q-values
            for experience in experiences:
                agent.update(*experience)
            
            score = env.calculate_timetable_score()
            if score > best_score:
                best_score = score
                best_timetable = copy.deepcopy(env.timetable)
                print(f"New best score: {best_score} at episode {episode}")
                
                # Save best timetable periodically
                if episode % save_interval == 0:
                    with open(f'best_timetable_ep_{episode}.pkl', 'wb') as f:
                        pickle.dump(best_timetable, f)
                    print(f"Saved best timetable at episode {episode}")
        
        if best_timetable is not None:
            print("\nGeneration complete! Displaying final timetable...")
            env.timetable = best_timetable
            env.display_detailed_timetable()
            
            # Export results
            try:
                env.save_timetable('final_timetable.json')
                print("Timetable saved to final_timetable.json")
            except Exception as e:
                print(f"Warning: Could not save timetable to JSON: {str(e)}")
            
            try:
                env.export_to_excel('timetable.xlsx')
                print("Timetable exported to timetable.xlsx")
            except Exception as e:
                print(f"Warning: Could not export to Excel: {str(e)}")
            
            # Calculate and display final metrics
            final_metrics = env.calculate_metrics()
            print("\nFinal Metrics:")
            print(json.dumps(final_metrics, indent=2))
            
            print("\nTimetable generation successful!")
            return True
        else:
            print("\nFailed to generate a valid timetable. Please try again with different parameters.")
            return False
            
    except Exception as e:
        print(f"\nError during timetable generation: {str(e)}")
        logging.error(f"Detailed error: {traceback.format_exc()}")
        return False

if __name__ == "__main__":
    success = main()
    if not success:
        print("\nTimetable generation encountered issues. Please check timetable_generator.log for details.")


2025-01-20 11:29:56,789 - INFO - No input file found, getting data from user input


Enter timetable parameters:

Enter class details:

Enter subject details:

Enter teacher details:

Enter details for teacher 1:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a1 (dhee) who can teach: DAA, DAALAB

Enter details for teacher 2:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a2 (spoo) who can teach: DAALAB, OS

Enter details for teacher 3:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a3 (bina) who can teach: OS, DAA

Enter requirements for class 1a:

For subject DAA:

Available teachers for this subject:
- a1: dhee
- a3: bina

Available teachers:
- a1: dhee
- a3: bina

Available teachers:
- a3: bina

Total lectures for DAA: 4
Teacher allocations:
- a1 (dhee): 2 lectures
- a3 (bina): 2 lectures

For subject DAALAB:

Available teachers for this subject:
- a1: dhee
- a2: spoo
Lab subject - Number of teachers set to number of batches (2)

Available teachers:
- a1: dhee
- a2: spoo
For labs,

2025-01-20 11:31:27,152 - INFO - Timetable saved to final_timetable.json
2025-01-20 11:31:27,162 - INFO - Timetable exported to timetable.xlsx


Training Episode 1500/3000...
Training Episode 2000/3000...
Training Episode 2500/3000...

Generation complete! Displaying final timetable...

Timetable for Class 1a
+-----------+------------------+------------------+-------------+------------+------------+
| Day       | Period 1         | Period 2         | Period 3    | Period 4   | Period 5   |
+===========+==================+==================+=============+============+============+
| Monday    | DAA              | DAA              | LUNCH BREAK | DAA        | DAA        |
|           | a1 - dhee        | a1 - dhee        |             | a3 - bina  | a3 - bina  |
+-----------+------------------+------------------+-------------+------------+------------+
| Tuesday   | DAALAB (Batch 1) | DAALAB (Batch 1) | LUNCH BREAK | OS         | OS         |
|           | a1 - dhee        | a1 - dhee        |             | a2 - spoo  | a2 - spoo  |
|           | ---              | ---              |             |            |            |
|     

In [12]:
import numpy as np
from collections import defaultdict
import random
import copy
from tabulate import tabulate
import traceback
import json
from datetime import datetime
import logging
from typing import Dict, List, Tuple, Optional, Any
import concurrent.futures
from dataclasses import dataclass
import pickle

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

@dataclass
class SubjectTeacherAllocation:
    teacher_code: str
    num_lectures: int
    assigned_lectures: int = 0
    
    def is_fully_allocated(self) -> bool:
        return self.assigned_lectures >= self.num_lectures

@dataclass
class BatchSchedule:
    subject: str
    teacher: str
    batch_num: Optional[int]
    
    def to_dict(self) -> Dict:
        return {
            'subject': self.subject,
            'teacher': self.teacher,
            'batch_num': self.batch_num
        }

class TimetableConstraints:
    def __init__(self):
        self.max_consecutive_same_subject = 2  # Maximum consecutive periods of same subject
        self.max_daily_lectures_per_teacher = 6
        self.preferred_lab_periods = [0, 2]
        self.min_gap_between_same_subject = 1
        self.max_subject_hours_per_day = 2  # Maximum periods of same subject per day

class MultiClassTimetableEnvironment:
    # ... [previous init and other methods remain the same]

    def __init__(self, days: int, periods_per_day: int, lunch_after_period: int,
                 classes: List[str], teachers: Dict[str, List[str]], 
                 teacher_names: Dict[str, str], subjects: Dict[str, Dict],
                 class_requirements: Dict[str, Dict], num_batches: int):
        self.days = days
        self.periods_per_day = periods_per_day
        self.lunch_after_period = lunch_after_period
        self.classes = classes
        self.teachers = teachers
        self.teacher_names = teacher_names
        self.subjects = subjects
        self.class_requirements = class_requirements
        self.num_batches = num_batches
        self.constraints = TimetableConstraints()
        
        # Initialize metrics tracking
        self.metrics = {
            'teacher_utilization': defaultdict(int),
            'subject_distribution': defaultdict(lambda: defaultdict(int)),
            'conflicts_avoided': 0
        }
        
        self.teacher_allocations = self._initialize_teacher_allocations()
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, classes[0])

    def load_timetable(self, filename: str):
        """Load a timetable state from a file"""
        with open(filename, 'r') as f:
            data = json.load(f)
            
        # Reconstruct timetable with proper objects
        for class_name, schedule in data['timetable'].items():
            for pos, slot in schedule.items():
                day, period = eval(pos)
                if isinstance(slot, list):
                    self.timetable[class_name][day, period] = [
                        BatchSchedule(**s) if isinstance(s, dict) else s
                        for s in slot
                    ]
                else:
                    self.timetable[class_name][day, period] = slot
                    
        self.metrics.update(data['metrics'])
        logging.info(f"Timetable loaded from {filename}")

    def _check_teacher_workload(self, day: int, teacher_code: str) -> bool:
        """Check if teacher has exceeded daily workload limit"""
        daily_lectures = 0
        for class_name in self.classes:
            for period in range(self.periods_per_day + 1):
                slot = self.timetable[class_name][day, period]
                if isinstance(slot, list):
                    for scheduled in slot:
                        if isinstance(scheduled, BatchSchedule) and scheduled.teacher == teacher_code:
                            daily_lectures += 1
        return daily_lectures < self.constraints.max_daily_lectures_per_teacher

    def _check_subject_spacing(self, day: int, period: int, subject: str, class_name: str) -> bool:
        """Check if subject spacing constraints are met"""
        # Check previous periods
        for p in range(max(0, period - self.constraints.min_gap_between_same_subject), period):
            slot = self.timetable[class_name][day, p]
            if isinstance(slot, list) and any(s.subject == subject for s in slot):
                return False
                
        # Check following periods
        for p in range(period + 1, 
                      min(self.periods_per_day + 1, 
                          period + self.constraints.min_gap_between_same_subject + 1)):
            slot = self.timetable[class_name][day, p]
            if isinstance(slot, list) and any(s.subject == subject for s in slot):
                return False
        return True

    def _initialize_teacher_allocations(self) -> Dict[str, Dict[str, List[SubjectTeacherAllocation]]]:
        """Initialize teacher allocations for each class and subject"""
        allocations = {}
        
        for class_name, requirements in self.class_requirements.items():
            allocations[class_name] = {}
            
            for subject, subject_reqs in requirements.items():
                if subject == 'teacher_code':  # Skip non-subject keys
                    continue
                    
                allocations[class_name][subject] = []
                teacher_allocs = subject_reqs.get('teacher_allocations', [])
                
                for teacher_code, num_lectures in teacher_allocs:
                    allocation = SubjectTeacherAllocation(
                        teacher_code=teacher_code,
                        num_lectures=num_lectures,
                        assigned_lectures=0
                    )
                    allocations[class_name][subject].append(allocation)
                    
                # Log initialization for debugging
                logging.debug(f"Initialized allocations for {class_name} - {subject}: "
                            f"{len(allocations[class_name][subject])} teachers")
        
        return allocations

    def _create_empty_timetable(self):
        """Create an empty timetable with support for multiple subjects per slot"""
        timetables = {}
        for class_name in self.classes:
            # Initialize with empty lists to store multiple subjects/batches
            timetable = np.empty((self.days, self.periods_per_day + 1), dtype=object)
            for day in range(self.days):
                for period in range(self.periods_per_day + 1):
                    timetable[day, period] = []
                    if period == self.lunch_after_period:
                        timetable[day, period] = 'LUNCH'
            timetables[class_name] = timetable
        return timetables

    def reset(self):
        """Reset the environment to initial state"""
        self.timetable = self._create_empty_timetable()
        self.current_position = (0, 0, self.classes[0])
        self.teacher_allocations = self._initialize_teacher_allocations()
        return self._get_state()
    
    def _can_schedule_lab(self, day, period, subject, class_name):
        """Check if a lab session can be scheduled for all batches"""
        if period >= self.periods_per_day - 1 or period == self.lunch_after_period - 1:
            return False
                
        current_slot = self.timetable[class_name][day, period]
        next_slot = self.timetable[class_name][day, period + 1]
            
        if current_slot == 'LUNCH' or next_slot == 'LUNCH':
            return False
                
        # Check if any batch has a lab at this time
        if current_slot or next_slot:
            return False
                
        # Get available teachers for this lab
        available_teachers = []
        allocations = self.teacher_allocations[class_name][subject]
        
        # For labs, total lectures across all teachers should match number of sessions
        total_assigned = sum(alloc.assigned_lectures for alloc in allocations)
        target_sessions = allocations[0].num_lectures  # All teachers should have same number
        
        if total_assigned >= target_sessions:
            return False
        
        # Check if we have enough available teachers for all batches
        for alloc in allocations:
            if self._is_teacher_available(day, period, alloc.teacher_code) and \
            self._is_teacher_available(day, period + 1, alloc.teacher_code):
                available_teachers.append(alloc.teacher_code)
        
        return len(available_teachers) >= self.num_batches

    def _is_teacher_available(self, day, period, teacher_code):
        """Check teacher availability across all classes and batches"""
        for class_name in self.classes:
            slot = self.timetable[class_name][day, period]
            if slot == 'LUNCH':
                continue
                
            for scheduled in slot:
                if isinstance(scheduled, BatchSchedule):
                    if scheduled.teacher == teacher_code:
                        return False
                else:  # Regular subject
                    if scheduled.teacher == teacher_code:
                        return False
        return True

    def _get_available_teacher(self, class_name, subject):
        """Get an available teacher with remaining lectures"""
        allocations = self.teacher_allocations[class_name][subject]
        for alloc in allocations:
            if alloc.assigned_lectures < alloc.num_lectures:
                return alloc.teacher_code
        return None

    def _update_teacher_allocation(self, class_name, subject, teacher_code):
        """Update the number of lectures assigned to a teacher"""
        for alloc in self.teacher_allocations[class_name][subject]:
            if alloc.teacher_code == teacher_code:
                alloc.assigned_lectures += 1
                break

    def _check_all_requirements_met(self):
        """Check if all teaching requirements have been met"""
        for class_name, subjects in self.teacher_allocations.items():
            for subject, allocations in subjects.items():
                for alloc in allocations:
                    if alloc.assigned_lectures < alloc.num_lectures:
                        return False
        return True

    def display_detailed_timetable(self):
        
        """Display the timetable with detailed information about teachers and batches"""
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        periods = [f'Period {i+1}' for i in range(self.periods_per_day + 1)]
        
        for class_name in self.classes:
            print(f"\n{'='*100}")
            print(f"Timetable for Class {class_name}")
            print(f"{'='*100}")
            
            table_data = []
            for day_idx, day in enumerate(days):
                row = [day]
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day_idx, period]
                    
                    if slot == 'LUNCH':
                        cell = 'LUNCH BREAK'
                    elif slot:
                        cell_parts = []
                        for scheduled in slot:
                            teacher_name = self.teacher_names[scheduled.teacher]
                            if scheduled.batch_num is not None:
                                cell_parts.append(
                                    f"{scheduled.subject} (Batch {scheduled.batch_num + 1})\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                            else:
                                cell_parts.append(
                                    f"{scheduled.subject}\n"
                                    f"{scheduled.teacher} - {teacher_name}"
                                )
                        cell = "\n---\n".join(cell_parts)
                    else:
                        cell = '-'
                    row.append(cell)
                table_data.append(row)
            
            print(tabulate(table_data, headers=['Day'] + periods, tablefmt='grid'))
            
            # Display teacher allocation summary
            print(f"\nTeacher Allocation Summary for Class {class_name}:")
            for subject, allocations in self.teacher_allocations[class_name].items():
                print(f"\n{subject}:")
                for alloc in allocations:
                    teacher_name = self.teacher_names[alloc.teacher_code]
                    print(f"- {alloc.teacher_code} ({teacher_name}):")
                    print(f"  Assigned: {alloc.assigned_lectures}/{alloc.num_lectures} lectures")
                    
    def _get_state(self):
    #"""Get current state of the environment"""
        day, period, current_class = self.current_position
        
        # Get remaining lectures for current class
        remaining_lectures = {}
        for subject, allocations in self.teacher_allocations[current_class].items():
            remaining = sum(alloc.num_lectures - alloc.assigned_lectures 
                        for alloc in allocations)
            remaining_lectures[subject] = remaining
        
        return {
            'position': self.current_position,
            'remaining_lectures': remaining_lectures
    }

    def calculate_timetable_score(self):
        """Calculate the score for the current timetable"""
        score = 0
        
        # Award points for each successfully scheduled lecture
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    score += alloc.assigned_lectures
        
        # Penalty for unassigned lectures
        for class_name in self.classes:
            for subject, allocations in self.teacher_allocations[class_name].items():
                for alloc in allocations:
                    unassigned = alloc.num_lectures - alloc.assigned_lectures
                    score -= unassigned * 2  # Higher penalty for unassigned lectures
        
        return score
        
        # Fix for the JSON saving method
        
    def save_timetable(self, filename: str):
        """Save the current timetable state to a file"""
        try:
            data = {
                'timetable': {},
                'metrics': dict(self.metrics),
                'timestamp': datetime.now().isoformat()
            }
            
            # Convert timetable data to serializable format
            for class_name in self.classes:
                data['timetable'][class_name] = {}
                for day in range(self.days):
                    for period in range(self.periods_per_day + 1):
                        slot = self.timetable[class_name][day, period]
                        key = f"{day},{period}"
                        
                        if slot == 'LUNCH':
                            data['timetable'][class_name][key] = 'LUNCH'
                        elif isinstance(slot, list):
                            data['timetable'][class_name][key] = [
                                {
                                    'subject': s.subject,
                                    'teacher': s.teacher,
                                    'batch_num': s.batch_num
                                } for s in slot if isinstance(s, BatchSchedule)
                            ]
                        else:
                            data['timetable'][class_name][key] = []
            
            with open(filename, 'w') as f:
                json.dump(data, f, indent=4)
            logging.info(f"Timetable saved to {filename}")
            
        except Exception as e:
            logging.error(f"Error saving timetable: {str(e)}")
            raise

    # Fix for the Excel export method
    def export_to_excel(self, filename: str):
        """Export timetable to Excel format"""
        try:
            import pandas as pd
            
            days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
            periods = [f'Period {p+1}' for p in range(self.periods_per_day + 1)]
            
            with pd.ExcelWriter(filename, engine='openpyxl') as writer:
                # Export timetable for each class
                for class_name in self.classes:
                    data = []
                    for day_idx, day in enumerate(days):
                        row = []
                        for period in range(self.periods_per_day + 1):
                            slot = self.timetable[class_name][day_idx, period]
                            if slot == 'LUNCH':
                                cell = 'LUNCH BREAK'
                            elif isinstance(slot, list) and slot:
                                cell_parts = []
                                for scheduled in slot:
                                    if isinstance(scheduled, BatchSchedule):
                                        teacher_name = self.teacher_names.get(scheduled.teacher, scheduled.teacher)
                                        if scheduled.batch_num is not None:
                                            cell_parts.append(
                                                f"{scheduled.subject} (B{scheduled.batch_num + 1})\n"
                                                f"{teacher_name}"
                                            )
                                        else:
                                            cell_parts.append(
                                                f"{scheduled.subject}\n{teacher_name}"
                                            )
                                cell = '\n'.join(cell_parts)
                            else:
                                cell = '-'
                            row.append(cell)
                        data.append(row)
                    
                    # Create DataFrame and write to Excel
                    df = pd.DataFrame(data, index=days, columns=periods)
                    df.to_excel(writer, sheet_name=f'Class_{class_name}')
                    
                    # Auto-adjust column widths
                    worksheet = writer.sheets[f'Class_{class_name}']
                    for idx, col in enumerate(df.columns):
                        max_length = max(
                            df[col].astype(str).apply(len).max(),
                            len(str(col))
                        )
                        worksheet.column_dimensions[chr(65 + idx + 1)].width = max_length + 2
                
                # Export metrics on a separate sheet
                metrics = self.calculate_metrics()
                metrics_df = pd.DataFrame([metrics])
                metrics_df.to_excel(writer, sheet_name='Metrics')
            
            logging.info(f"Timetable exported to {filename}")
            
        except ImportError:
            logging.error("Pandas is required for Excel export. Please install it first.")
            raise
        except Exception as e:
            logging.error(f"Error exporting to Excel: {str(e)}")
            raise

    # Also add these imports at the top of the file if not already present
    def get_input_data():
        """Get all necessary input data from user"""
        print("Enter timetable parameters:")
        days = 5  # Fixed to 5 days
        periods_per_day = int(input("Number of periods per day: "))
        lunch_after_period = int(input("Lunch break after period number: "))
        
        print("\nEnter class details:")
        num_classes = int(input("Number of classes: "))
        classes = []
        for i in range(num_classes):
            class_name = input(f"Name for class {i+1}: ")
            classes.append(class_name)
        
        print("\nEnter subject details:")
        subjects = {}
        num_subjects = int(input("Number of subjects: "))
        for i in range(num_subjects):
            while True:
                subject_name = input(f"Name for subject {i+1}: ").upper()
                if subject_name in subjects:
                    print("Subject already exists. Please enter a different name.")
                    continue
                break
            is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
            subjects[subject_name] = {'is_lab': is_lab}
        
        num_batches = int(input("\nNumber of lab batches: "))
        
        print("\nEnter teacher details:")
        teachers, teacher_names = get_teacher_details()
        
        class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names, num_batches)
        
        return (days, periods_per_day, lunch_after_period, classes, teachers, 
                teacher_names, subjects, class_requirements, num_batches)
    
    def _check_consecutive_subject_constraint(self, day: int, period: int, subject: str, class_name: str) -> bool:
        """
        Check if scheduling this subject would violate the consecutive subject constraint
        Returns True if constraint is satisfied, False if violated
        """
        # Check previous periods for consecutive subjects
        consecutive_count = 0
        
        # Look backwards up to 2 periods
        for p in range(period - 1, max(-1, period - 3), -1):
            if p < 0 or self.timetable[class_name][day, p] == 'LUNCH':
                break
                
            slot = self.timetable[class_name][day, p]
            if isinstance(slot, list) and slot:
                prev_subject = slot[0].subject  # Get subject from first schedule in the slot
                if prev_subject == subject:
                    consecutive_count += 1
                else:
                    break
                    
        # Look forward one period (if not at end)
        if period < self.periods_per_day:
            next_slot = self.timetable[class_name][day, period + 1]
            if isinstance(next_slot, list) and next_slot:
                next_subject = next_slot[0].subject
                if next_subject == subject:
                    consecutive_count += 1
                    
        return consecutive_count < self.constraints.max_consecutive_same_subject

    def _check_daily_subject_limit(self, day: int, subject: str, class_name: str) -> bool:
        """
        Check if the subject has reached its daily limit
        Returns True if within limit, False if exceeded
        """
        daily_count = 0
        for period in range(self.periods_per_day + 1):
            slot = self.timetable[class_name][day, period]
            if isinstance(slot, list) and slot:
                for scheduled in slot:
                    if isinstance(scheduled, BatchSchedule) and scheduled.subject == subject:
                        daily_count += 1
                        
        return daily_count < self.constraints.max_subject_hours_per_day

    def step(self, action):
        """Take a step in the environment with updated constraints"""
        day, period, current_class = self.current_position
        subject, is_lab = action
        reward = 0
        done = False
        
        if period == self.lunch_after_period:
            period += 1
        
        # Check consecutive subject constraint
        if not self._check_consecutive_subject_constraint(day, period, subject, current_class):
            reward = -1
            logging.debug(f"Consecutive subject constraint violated: {subject} at day {day}, period {period}")
        # Check daily subject limit
        elif not self._check_daily_subject_limit(day, subject, current_class):
            reward = -1
            logging.debug(f"Daily subject limit exceeded: {subject} at day {day}")
        else:
            if is_lab:
                # Lab scheduling logic remains the same
                if self._can_schedule_lab(day, period, subject, current_class):
                    available_teachers = []
                    allocations = self.teacher_allocations[current_class][subject]
                    for alloc in allocations:
                        if self._is_teacher_available(day, period, alloc.teacher_code):
                            available_teachers.append(alloc.teacher_code)
                    
                    if len(available_teachers) >= self.num_batches:
                        # Schedule lab for all batches
                        for batch in range(self.num_batches):
                            teacher_code = available_teachers[batch]
                            batch_schedule = BatchSchedule(subject, teacher_code, batch)
                            self.timetable[current_class][day, period].append(batch_schedule)
                            self.timetable[current_class][day, period + 1].append(batch_schedule)
                        
                        # Update allocations
                        for alloc in allocations:
                            alloc.assigned_lectures += 1
                        
                        reward = 1
                        period += 1
                    else:
                        reward = -1
                else:
                    reward = -1
            else:
                # Regular lecture scheduling
                teacher_code = self._get_available_teacher(current_class, subject)
                if teacher_code and self._is_teacher_available(day, period, teacher_code):
                    self.timetable[current_class][day, period].append(
                        BatchSchedule(subject, teacher_code, None)
                    )
                    self._update_teacher_allocation(current_class, subject, teacher_code)
                    reward = 1
                else:
                    reward = -1
        
        # Update position and check if done
        class_idx = self.classes.index(current_class)
        class_idx += 1
        if class_idx >= len(self.classes):
            class_idx = 0
            period += 1
            if period >= self.periods_per_day + 1:
                period = 0
                day += 1
        
        if day >= self.days:
            done = True
            if self._check_all_requirements_met():
                reward += 100
        
        self.current_position = (day, period, self.classes[class_idx])
        return self._get_state(), reward, done

    def calculate_metrics(self):
        """Calculate comprehensive metrics including constraint violations"""
        metrics = super().calculate_metrics()
        
        # Add constraint violation metrics
        constraint_violations = {
            'consecutive_subject_violations': 0,
            'daily_subject_limit_violations': 0
        }
        
        for class_name in self.classes:
            for day in range(self.days):
                # Check consecutive subject violations
                for period in range(self.periods_per_day - 1):
                    subject_sequence = []
                    for p in range(period, min(period + 3, self.periods_per_day + 1)):
                        slot = self.timetable[class_name][day, p]
                        if isinstance(slot, list) and slot:
                            subject_sequence.append(slot[0].subject)
                            
                    if len(subject_sequence) >= 3:
                        if all(s == subject_sequence[0] for s in subject_sequence):
                            constraint_violations['consecutive_subject_violations'] += 1
                
                # Check daily subject limit violations
                subject_counts = defaultdict(int)
                for period in range(self.periods_per_day + 1):
                    slot = self.timetable[class_name][day, period]
                    if isinstance(slot, list) and slot:
                        for scheduled in slot:
                            subject_counts[scheduled.subject] += 1
                            
                for subject, count in subject_counts.items():
                    if count > self.constraints.max_subject_hours_per_day:
                        constraint_violations['daily_subject_limit_violations'] += 1
        
        metrics['constraint_violations'] = constraint_violations
        return metrics
     
class ParallelQLearningAgent(QLearningAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_workers = 4
        
    def parallel_update(self, experiences):
        """Update Q-values in parallel"""
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.num_workers) as executor:
            futures = []
            for state, action, reward, next_state in experiences:
                futures.append(
                    executor.submit(self.update, state, action, reward, next_state)
                )
            concurrent.futures.wait(futures)

def get_input_data_from_file(filename: str) -> Tuple:
    """Load input data from JSON file"""
    with open(filename, 'r') as f:
        data = json.load(f)
    
    required_fields = ['days', 'periods_per_day', 'lunch_after_period', 'classes',
                      'teachers', 'teacher_names', 'subjects', 'class_requirements',
                      'num_batches']
                      
    for field in required_fields:
        if field not in data:
            raise ValueError(f"Missing required field: {field}")
            
    return (data['days'], data['periods_per_day'], data['lunch_after_period'],
            data['classes'], data['teachers'], data['teacher_names'],
            data['subjects'], data['class_requirements'], data['num_batches'])

class QLearningAgent:
    def __init__(self, action_space, learning_rate=0.1, discount_factor=0.95, epsilon=0.3):
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.action_space = action_space
    
    def get_state_key(self, state):
        """Convert state to a string key for Q-table"""
        if state and 'position' in state:
            return str(state['position'])
        return "default"
    
    def get_action(self, state, valid_actions):
        """Get action using epsilon-greedy policy"""
        if not valid_actions:
            return None
        
        if random.random() < self.epsilon:
            return random.choice(valid_actions)
        
        state_key = self.get_state_key(state)
        q_values = {action: self.q_table[state_key][action] 
                   for action in valid_actions}
        
        return max(q_values.items(), key=lambda x: x[1])[0]
    
    def update(self, state, action, reward, next_state):
        """Update Q-value for state-action pair"""
        if not state or not next_state or not action:
            return
        
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)
        
        current_q = self.q_table[state_key][action]
        next_max_q = max([self.q_table[next_state_key][a] for a in self.action_space])
        
        new_q = current_q + self.lr * (reward + self.gamma * next_max_q - current_q)
        self.q_table[state_key][action] = new_q

def get_teacher_details():
    """Get teacher details from user input with better subject handling"""
    teachers = {}
    teacher_names = {}
    
    num_teachers = int(input("Number of teachers: "))
    
    for i in range(num_teachers):
        print(f"\nEnter details for teacher {i+1}:")
        code = input("Teacher code (e.g., T001): ")
        name = input("Teacher name: ")
        print("Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): ")
        subjects = [s.strip().upper() for s in input().split(',')]
        
        teachers[code] = subjects
        teacher_names[code] = name
        print(f"Added teacher {code} ({name}) who can teach: {', '.join(subjects)}")
    
    return teachers, teacher_names

def get_class_requirements(classes, subjects, teachers, teacher_names, num_batches):
    """Modified input validation for lab teacher requirements"""
    class_requirements = {}
    
    for class_name in classes:
        print(f"\nEnter requirements for class {class_name}:")
        class_requirements[class_name] = {}
        
        for subject_name, subject_info in subjects.items():
            print(f"\nFor subject {subject_name}:")
            
            # Show available teachers for this subject
            available_teachers = [
                (code, name) for code, name in teacher_names.items()
                if subject_name.upper() in [s.upper() for s in teachers[code]]
            ]
            
            if not available_teachers:
                print(f"Warning: No teachers available for {subject_name}")
                continue
                
            print("\nAvailable teachers for this subject:")
            for code, name in available_teachers:
                print(f"- {code}: {name}")
            
            # For labs, enforce number of teachers equals number of batches
            if subject_info['is_lab']:
                num_teachers = num_batches
                print(f"Lab subject - Number of teachers set to number of batches ({num_batches})")
            else:
                while True:
                    try:
                        num_teachers = int(input(f"Number of teachers for {subject_name}: "))
                        if num_teachers <= 0:
                            print("Please enter a positive number")
                            continue
                        if num_teachers > len(available_teachers):
                            print(f"Only {len(available_teachers)} teachers available")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
            
            teacher_allocations = []
            total_lectures = 0
            
            # Get allocation for each teacher
            assigned_teachers = set()
            for i in range(num_teachers):
                while True:
                    print("\nAvailable teachers:")
                    for code, name in available_teachers:
                        if code not in assigned_teachers:
                            print(f"- {code}: {name}")
                            
                    teacher_code = input(f"Select teacher {i+1} code for {subject_name}: ")
                    
                    if teacher_code not in teachers:
                        print("Invalid teacher code")
                        continue
                    if teacher_code in assigned_teachers:
                        print("This teacher is already assigned")
                        continue
                    if subject_name.upper() not in [s.upper() for s in teachers[teacher_code]]:
                        print(f"Teacher {teacher_code} cannot teach {subject_name}")
                        continue
                    break
                
                while True:
                    try:
                        if subject_info['is_lab']:
                            if i == 0:  # Only ask once for labs
                                print("For labs, enter total number of lab sessions")
                                print("(This will be shared among all teachers)")
                                lectures = int(input(f"Number of lab sessions: "))
                            # All teachers get the same number for labs
                        else:
                            lectures = int(input(f"Number of lectures for teacher {teacher_code}: "))
                        
                        if lectures <= 0:
                            print("Please enter a positive number")
                            continue
                        if subject_info['is_lab'] and lectures % 2 != 0:
                            print("Lab sessions must be even number")
                            continue
                        break
                    except ValueError:
                        print("Please enter a valid number")
                
                teacher_allocations.append((teacher_code, lectures))
                assigned_teachers.add(teacher_code)
                
                if subject_info['is_lab']:
                    # For labs, all teachers share the same total
                    total_lectures = lectures
                else:
                    total_lectures += lectures
                
            class_requirements[class_name][subject_name] = {
                'teacher_allocations': teacher_allocations,
                'total_lectures': total_lectures
            }
            
            print(f"\nTotal lectures for {subject_name}: {total_lectures}")
            print("Teacher allocations:")
            for teacher_code, lectures in teacher_allocations:
                print(f"- {teacher_code} ({teacher_names[teacher_code]}): "
                      f"{lectures} {'sessions' if subject_info['is_lab'] else 'lectures'}")
    
    return class_requirements

def get_input_data():
    """Get all necessary input data from user"""
    print("Enter timetable parameters:")
    days = 5  # Fixed to 5 days
    periods_per_day = int(input("Number of periods per day: "))
    lunch_after_period = int(input("Lunch break after period number: "))
    
    print("\nEnter class details:")
    num_classes = int(input("Number of classes: "))
    classes = []
    for i in range(num_classes):
        class_name = input(f"Name for class {i+1}: ")
        classes.append(class_name)
    
    print("\nEnter subject details:")
    subjects = {}
    num_subjects = int(input("Number of subjects: "))
    for i in range(num_subjects):
        while True:
            subject_name = input(f"Name for subject {i+1}: ").upper()
            if subject_name in subjects:
                print("Subject already exists. Please enter a different name.")
                continue
            break
        is_lab = input(f"Is {subject_name} a lab subject? (yes/no): ").lower() == 'yes'
        subjects[subject_name] = {'is_lab': is_lab}
    
    num_batches = int(input("\nNumber of lab batches: "))
    
    print("\nEnter teacher details:")
    teachers, teacher_names = get_teacher_details()
    
    class_requirements = get_class_requirements(classes, subjects, teachers, teacher_names, num_batches)
    
    return (days, periods_per_day, lunch_after_period, classes, teachers, 
            teacher_names, subjects, class_requirements, num_batches)

def train_timetable_generator(env, agent, episodes=1000):
    """Train the timetable generator"""
    best_timetable = None
    best_score = float('-inf')
    
    for episode in range(episodes):
        state = env.reset() if hasattr(env, 'reset') else None
        total_reward = 0
        done = False
        
        while not done:
            if not state:
                break
            
            current_class = state['position'][2]
            valid_actions = []
            
            for action in agent.action_space:
                subject, is_lab = action
                if env._get_available_teacher(current_class, subject):
                    valid_actions.append(action)
            
            if not valid_actions:
                break
            
            action = agent.get_action(state, valid_actions)
            if not action:
                break
            
            next_state, reward, done = env.step(action)
            agent.update(state, action, reward, next_state)
            
            state = next_state
            total_reward += reward
        
        score = env.calculate_timetable_score() if hasattr(env, 'calculate_timetable_score') else total_reward
        if score > best_score:
            best_score = score
            best_timetable = copy.deepcopy(env.timetable)
        
        if episode % 100 == 0:
            print(f"Episode {episode}, Best Score: {best_score}")
    
    return best_timetable

def main():
    try:
        # Set up logging to both file and console
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('timetable_generator.log'),
                logging.StreamHandler()  # This will show logs in console
            ]
        )
        
        # Get input data
        try:
            input_data = get_input_data_from_file('input_data.json')
            logging.info("Input data loaded from file")
        except FileNotFoundError:
            logging.info("No input file found, getting data from user input")
            input_data = get_input_data()
            
        # Create environment
        env = MultiClassTimetableEnvironment(*input_data)
        
        # Create action space
        action_space = [
            (subject, info['is_lab'])
            for subject, info in input_data[6].items()  # subjects dict
        ]
        
        # Create and train agent with improved parameters
        agent = ParallelQLearningAgent(
            action_space,
            learning_rate=0.15,
            discount_factor=0.99,
            epsilon=0.4
        )
        
        # Train with periodic saving
        best_timetable = None
        best_score = float('-inf')
        
        num_episodes = 3000
        save_interval = 500
        
        for episode in range(num_episodes):
            if episode % save_interval == 0:
                print(f"Training Episode {episode}/{num_episodes}...")
            
            state = env.reset()
            experiences = []
            total_reward = 0
            done = False
            
            while not done:
                if not state:
                    break
                    
                current_class = state['position'][2]
                valid_actions = [
                    action for action in agent.action_space
                    if env._get_available_teacher(current_class, action[0])
                ]
                
                if not valid_actions:
                    break
                    
                action = agent.get_action(state, valid_actions)
                if not action:
                    break
                    
                next_state, reward, done = env.step(action)
                experiences.append((state, action, reward, next_state))
                
                state = next_state
                total_reward += reward
            
            # Update Q-values
            for experience in experiences:
                agent.update(*experience)
            
            score = env.calculate_timetable_score()
            if score > best_score:
                best_score = score
                best_timetable = copy.deepcopy(env.timetable)
                print(f"New best score: {best_score} at episode {episode}")
                
                # Save best timetable periodically
                if episode % save_interval == 0:
                    with open(f'best_timetable_ep_{episode}.pkl', 'wb') as f:
                        pickle.dump(best_timetable, f)
                    print(f"Saved best timetable at episode {episode}")
        
        if best_timetable is not None:
            print("\nGeneration complete! Displaying final timetable...")
            env.timetable = best_timetable
            env.display_detailed_timetable()
            
            # Export results
            try:
                env.save_timetable('final_timetable.json')
                print("Timetable saved to final_timetable.json")
            except Exception as e:
                print(f"Warning: Could not save timetable to JSON: {str(e)}")
            
            try:
                env.export_to_excel('timetable.xlsx')
                print("Timetable exported to timetable.xlsx")
            except Exception as e:
                print(f"Warning: Could not export to Excel: {str(e)}")
            
            # Calculate and display final metrics
            final_metrics = env.calculate_metrics()
            print("\nFinal Metrics:")
            print(json.dumps(final_metrics, indent=2))
            
            print("\nTimetable generation successful!")
            return True
        else:
            print("\nFailed to generate a valid timetable. Please try again with different parameters.")
            return False
            
    except Exception as e:
        print(f"\nError during timetable generation: {str(e)}")
        logging.error(f"Detailed error: {traceback.format_exc()}")
        return False

if __name__ == "__main__":
    success = main()
    if not success:
        print("\nTimetable generation encountered issues. Please check timetable_generator.log for details.")


2025-01-20 11:57:12,701 - INFO - No input file found, getting data from user input


Enter timetable parameters:

Enter class details:

Enter subject details:

Enter teacher details:

Enter details for teacher 1:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a1 (dhee) who can teach: DAA, DAALAB

Enter details for teacher 2:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a2 (spoo) who can teach: DAALAB, OS

Enter details for teacher 3:
Enter subjects taught (comma-separated, e.g., MATH,PHYSICS): 
Added teacher a3 (bina) who can teach: DAALAB, OS, DAA

Enter requirements for class 1a:

For subject DAA:

Available teachers for this subject:
- a1: dhee
- a3: bina

Available teachers:
- a1: dhee
- a3: bina

Available teachers:
- a3: bina

Total lectures for DAA: 4
Teacher allocations:
- a1 (dhee): 2 lectures
- a3 (bina): 2 lectures

For subject DAALAB:

Available teachers for this subject:
- a1: dhee
- a2: spoo
- a3: bina
Lab subject - Number of teachers set to number of batches (3)

Available teachers:
- a1: dhee
-

2025-01-20 11:58:43,631 - INFO - Timetable saved to final_timetable.json
2025-01-20 11:58:43,638 - ERROR - Error exporting to Excel: 'super' object has no attribute 'calculate_metrics'
2025-01-20 11:58:43,639 - ERROR - Detailed error: Traceback (most recent call last):
  File "/var/folders/51/n1l2lqnd2f3fl7dd3vfgm5mh0000gn/T/ipykernel_8686/3924129175.py", line 1040, in main
    final_metrics = env.calculate_metrics()
  File "/var/folders/51/n1l2lqnd2f3fl7dd3vfgm5mh0000gn/T/ipykernel_8686/3924129175.py", line 605, in calculate_metrics
    metrics = super().calculate_metrics()
AttributeError: 'super' object has no attribute 'calculate_metrics'




Generation complete! Displaying final timetable...

Timetable for Class 1a
+-----------+------------------+------------------+-------------+------------+------------+
| Day       | Period 1         | Period 2         | Period 3    | Period 4   | Period 5   |
+===========+==================+==================+=============+============+============+
| Monday    | DAA              | OS               | LUNCH BREAK | DAA        | -          |
|           | a1 - dhee        | a3 - bina        |             | a1 - dhee  |            |
+-----------+------------------+------------------+-------------+------------+------------+
| Tuesday   | DAALAB (Batch 1) | DAALAB (Batch 1) | LUNCH BREAK | -          | DAA        |
|           | a1 - dhee        | a1 - dhee        |             |            | a3 - bina  |
|           | ---              | ---              |             |            |            |
|           | DAALAB (Batch 2) | DAALAB (Batch 2) |             |            |            |
|   